# 1. Librería

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
#from utils.Validator import *
import pickle
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# 2. Importación de la data

In [2]:
train = pd.read_csv(r"../data/modeling/03_train_ppjj.csv",sep="|")   
test = pd.read_csv(r"../data/modeling/03_test_ppjj.csv",sep="|") 

## Train

In [3]:
train_sc2=train[(train.MTO_INGRESOS_MES>=1100)&(train.MTO_INGRESOS_MES<=30e+6)]

In [4]:
train_sc3 = train[train['MTO_INGRESOS_MES']>30e+6]
train_sc3.shape

(76, 43)

In [5]:
train_sc3['n_cluster']=1

## Test

In [6]:
test_sc2=test[(test.MTO_INGRESOS_MES>=1100)&(test.MTO_INGRESOS_MES<=30e+6)]

In [7]:
test_sc3 = test[test['MTO_INGRESOS_MES']>30e+6]
test_sc3.shape

(13, 43)

In [8]:
test_sc3['n_cluster']=1

# 2. Exportación del modelo

In [9]:
model = pickle.load(open(r"../src/02_models/iforest_model_ppjj", 'rb'))

# 3. Exportación de data

In [10]:
x_model=['FLG_PERFIL_INGRESOS_3DS','MTO_CONOTROSPROXIMOS', 'CTD_CONOTROSPROXIMOS']

In [11]:
test.columns

Index(['PERIODO', 'CODCLAVECIC', 'EDAD', 'TIPPER', 'ANTIGUEDAD',
       'CODACTECONOMICA', 'DESACTECONOMICA', 'FLG_ACTECO_NODEF', 'FLGNP',
       'CTDNP', 'FLGLSB', 'CTDLSB', 'FLGARCHIVONEGATIVO',
       'DESTIPMOTIVONEGATIVO', 'CTD_NP_LSB', 'CTDEVAL', 'CODSUBSEGMENTO',
       'DESSUBSEGMENTO', 'CODSEGMENTO', 'DESSEGMENTO', 'TIPDIR', 'CODUBIGEO',
       'CODDISTRITO', 'DESCODDISTRITO', 'CODPROVINCIA', 'DESCODPROVINCIA',
       'CODDEPARTAMENTO', 'DESCODDEPARTAMENTO', 'CODPAISNACIONALIDAD',
       'DESCODPAISNACIONALIDAD', 'FLGNACIONALIDAD', 'MTO_INGRESOS_MES',
       'CTD_INGRESOS_MES', 'MEDIA_INGRESOS', 'DESV_INGRESOS',
       'FLG_PERFIL_INGRESOS_3DS', 'MAX_CTDMTOSREDONDOS', 'SUM_MTOSREDONDOS',
       'CTDMAXIMA', 'MTO_MAXIMOSREPETIDOS', 'MTOMAXDEMAXREPETIDOS',
       'MTO_CONOTROSPROXIMOS', 'CTD_CONOTROSPROXIMOS'],
      dtype='object')

In [12]:
vars_export=['PERIODO', 'CODCLAVECIC','TIPPER','MTO_INGRESOS_MES','FLG_PERFIL_INGRESOS_3DS','MTO_CONOTROSPROXIMOS', 'CTD_CONOTROSPROXIMOS','n_cluster']

In [13]:
# TRAIN
train_sc2['n_cluster']=model.predict(train_sc2[x_model])
#Alertas según el modelo Isolations 
alertas_train=train_sc2[train_sc2["n_cluster"]==-1]
alertas_train=pd.concat([alertas_train, train_sc3], ignore_index=True)
#Unimos las alertas del escalón 3 y 2
alertas_train[vars_export].to_csv(r"../data/modeling/04_predicted_train_ppjj.csv",index=False,sep="|")

In [14]:
# TEST
test_sc2['n_cluster']=model.predict(test_sc2[x_model])
#Alertas según el modelo Isolations 
alertas_test=test_sc2[test_sc2["n_cluster"]==-1]
alertas_test=pd.concat([alertas_test, test_sc3], ignore_index=True)
#Unimos las alertas del escalón 3 y 2
alertas_test[vars_export].to_csv(r"../data/modeling/04_predicted_test_ppjj.csv",index=False,sep="|")

In [15]:
alertas_test.shape # 13 modelo - 13 directo a eval
# -1 modelo
# 1 ouliers (DaE)

(26, 44)